In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
TRAIN_DIR = './HGR/train/'
TEST_DIR = './HGR/test/'
IMG_SIZE = (240, 640)
BATCH_SIZE = 8
SEED = 1337

In [3]:
# Training generator
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_ds = train_datagen.flow_from_directory(TRAIN_DIR,
                                             target_size=IMG_SIZE,
                                             batch_size=BATCH_SIZE,
                                             seed=SEED,
                                             shuffle=True,
                                             class_mode='categorical')

# Test generator
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_ds = test_datagen.flow_from_directory(TEST_DIR,
                                           target_size=IMG_SIZE,
                                           batch_size=BATCH_SIZE,
                                           seed=SEED,
                                           shuffle=False,
                                           class_mode='categorical')

train_num = train_ds.samples
test_num = test_ds.samples

Found 16000 images belonging to 10 classes.
Found 4000 images belonging to 10 classes.


In [4]:
mod = Sequential()
mod.add(Conv2D(filters=32, kernel_size=(5, 5), input_shape=(240, 640, 3), padding='same', activation = 'relu'))
mod.add(MaxPool2D(2, 2))
mod.add(Conv2D(32, (3, 3), activation = 'relu'))
mod.add(MaxPool2D(2, 2))
mod.add(Conv2D(64, (3, 3), activation = 'relu'))
mod.add(Conv2D(64, (3, 3), activation = 'relu'))
mod.add(Conv2D(128, (3, 3), activation = 'relu'))
mod.add(MaxPool2D(2, 2))
mod.add(GlobalAvgPool2D())
mod.add(Dropout(0.3))

mod.add(Flatten())
mod.add(Activation('relu'))
mod.add(Dense(10))
mod.add(Activation('softmax'))

mod.summary()

mod.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 240, 640, 32)      2432      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 120, 320, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 118, 318, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 59, 159, 32)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 57, 157, 64)       18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 55, 155, 64)       3

In [5]:
EPOCHS = 5

checkpoint_path = "checkpoint/test1/cp.ckpt"

# mod.load_weights(checkpoint_path)

cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              save_weights_only=True,
                              save_best_only=True,
                              mode='min',
                              verbose=1)

history = mod.fit(train_ds,
                              steps_per_epoch=train_num // BATCH_SIZE,
                              epochs=EPOCHS,
                              validation_data=test_ds,
                              validation_steps=test_num // BATCH_SIZE,
                              callbacks=[cp_callback])

Epoch 1/5
2000/2000 [==============================] - ETA: 0s - loss: 2.1368 - accuracy: 0.2065
Epoch 1: val_loss improved from inf to 1.66389, saving model to checkpoint/test1\cp.ckpt
2000/2000 [==============================] - 1848s 923ms/step - loss: 2.1368 - accuracy: 0.2065 - val_loss: 1.6639 - val_accuracy: 0.5422
Epoch 2/5
2000/2000 [==============================] - ETA: 0s - loss: 1.6240 - accuracy: 0.4251
Epoch 2: val_loss improved from 1.66389 to 1.01045, saving model to checkpoint/test1\cp.ckpt
2000/2000 [==============================] - 1914s 957ms/step - loss: 1.6240 - accuracy: 0.4251 - val_loss: 1.0104 - val_accuracy: 0.6407
Epoch 3/5
2000/2000 [==============================] - ETA: 0s - loss: 1.2515 - accuracy: 0.5596
Epoch 3: val_loss improved from 1.01045 to 0.75413, saving model to checkpoint/test1\cp.ckpt
2000/2000 [==============================] - 1962s 981ms/step - loss: 1.2515 - accuracy: 0.5596 - val_loss: 0.7541 - val_accuracy: 0.8037
Epoch 4/5
2000/2000 

In [6]:
checkpoint_path = "checkpoint/test1/cp.ckpt"

mod.load_weights(checkpoint_path)

mod.evaluate(test_ds, batch_size=BATCH_SIZE)

mod.save('GR_net.h5')

500/500 [==============================] - 99s 199ms/step - loss: 0.6929 - accuracy: 0.8085


In [ ]:
mod = tf.keras.models.load_model('GR_net.h5')

mod.evaluate(test_ds, batch_size=BATCH_SIZE)